In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda

In [ ]:
# Load the conformational ensemble
state_name = 'active'
state_name = 'inactive'
u = mda.Universe(f'data/unbiased/{state_name}.pdb',
                 f'data/unbiased/{state_name}.xtc')

In [13]:
# Load the reference structure
ref = mda.Universe(f'test/{state_name}_abfe/equil/pose0/full.pdb',
                   f'test/{state_name}_abfe/equil/pose0/full.inpcrd', format='RESTRT')

In [ ]:
gpcr_sel = 'protein and chainID R and name CA'
gpcr_atoms = u.select_atoms(gpcr_sel)
print(gpcr_atoms.n_atoms)

In [15]:
from MDAnalysis.analysis import rms, align

average = align.AverageStructure(u, u, select=gpcr_sel,
                                 ref_frame=0).run()
ref = average.results.universe

aligner = align.AlignTraj(u, ref,
                          select=gpcr_sel + ' and resid 60-150',
                          in_memory=True).run()

In [16]:
R = rms.RMSF(gpcr_atoms).run()

In [ ]:
plt.plot(gpcr_atoms.resids, R.results.rmsf, label=state_name)
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.legend()

In [ ]:
# store the average position
ref.atoms.write(f'test/{state_name}_avg.pdb')

In [19]:
# save rmsf
rmsf_values = R.results.rmsf
with open(f'test/{state_name}_rmsf.txt', 'w') as f:
    for resid, rmsf in zip(gpcr_atoms.resids, rmsf_values):
        f.write(f'{resid} {rmsf}\n')

In [ ]:
!cat test/inactive_rmsf.txt